In [15]:
import os
import shutil

train_filenames = os.listdir('train')
train_cat = filter(lambda x:x[:3] == 'cat', train_filenames)
train_dog = filter(lambda x:x[:3] == 'dog', train_filenames)

def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

rmrf_mkdir('train2')
os.mkdir('train2/cat')
os.mkdir('train2/dog')

rmrf_mkdir('test2')
os.symlink('../test/', 'test2/test')

for filename in train_cat:
    os.symlink('../../train/'+filename, 'train2/cat/'+filename)

for filename in train_dog:
    os.symlink('../../train/'+filename, 'train2/dog/'+filename)

In [ ]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import h5py

def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("train2", image_size, shuffle=False, 
                                              batch_size=16)
    test_generator = gen.flow_from_directory("test2", image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)

    nb_sample = len(train_generator.filenames)
    print('nb_sample:',nb_sample)
    train = model.predict_generator(train_generator, nb_sample)
    test = model.predict_generator(test_generator, nb_sample)
    with h5py.File("gap_%s.h5"%MODEL.func_name) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

write_gap(ResNet50, (224, 224))
write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input)
write_gap(Xception, (299, 299), xception.preprocess_input)

Found 24722 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
nb_sample: 24722


In [5]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

for filename in ["../saved_models/gap_ResNet50.h5", "../saved_models/gap_Xception.h5", "../saved_models/gap_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

In [6]:
from keras.models import *
from keras.layers import *

np.random.seed(2017)

input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [11]:
# digraph G{
#     node [shape=record]
#     a[label="ResNet50|{input:|output:}|{(224, 224, 3)|(2048)}"]
#     b[label="InceptionV3|{input:|output:}|{(299, 299, 3)|(2048)}"]
#     c[label="Xception|{input:|output:}|{(299, 299, 3)|(2048)}"]
#     Merge[label="Merge|{input:|output:}|{(3, 2048)|(6144)}"]
#     Dropout[label="Dropout|Rate:|0.5"]
#     Output[label="Output|{input:|output:}|{(6144)|(1)}"]
#     Image -> a -> Merge
#     Image -> b -> Merge
#     Image -> c -> Merge
#     Merge -> Dropout -> Output
# }

In [12]:
model.fit(X_train, y_train, batch_size=128, nb_epoch=8, validation_split=0.2)

/home/ubuntu/anaconda3/envs/dog-project/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 1s - loss: 0.1182 - acc: 0.9609 - val_loss: 0.0286 - val_acc: 0.9922
Epoch 2/8
20000/20000 [==============================] - 1s - loss: 0.0335 - acc: 0.9888 - val_loss: 0.0168 - val_acc: 0.9952
Epoch 3/8
20000/20000 [==============================] - 1s - loss: 0.0250 - acc: 0.9916 - val_loss: 0.0141 - val_acc: 0.9958
Epoch 4/8
20000/20000 [==============================] - 1s - loss: 0.0224 - acc: 0.9928 - val_loss: 0.0133 - val_acc: 0.9954
Epoch 5/8
20000/20000 [==============================] - 1s - loss: 0.0206 - acc: 0.9937 - val_loss: 0.0119 - val_acc: 0.9964
Epoch 6/8
20000/20000 [==============================] - 1s - loss: 0.0177 - acc: 0.9941 - val_loss: 0.0113 - val_acc: 0.9960
Epoch 7/8
20000/20000 [==============================] - 1s - loss: 0.0171 - acc: 0.9944 - val_loss: 0.0111 - val_acc: 0.9960
Epoch 8/8
20000/20000 [==============================] - 1s - loss: 0

In [14]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("../sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test2", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('pred.csv', index=None)
df.head(10)

10784/12500 [========================>.....] - ETA: 0sFound 12500 images belonging to 1 classes.


,id,label
0,1,0.995000
1,2,0.995000
2,3,0.005000
3,4,0.995000
4,5,0.005000
5,6,0.005000
6,7,0.995000
7,8,0.995000
8,9,0.005000
9,10,0.562013
